In [81]:
# anchors = [[5,5,5],[10,10,10],[15,15,15],[20,20,20],[30,30,30],[5,5,2.5],[10,10,5],[15,15,7.5],[20,20,10],[30,30,15]]
# anchors = [[5,5,5],[10,10,10],[15,15,15],[20,20,20],[30,30,30]]
anchors = [[6,6,6],[12,12,12],[24,24,24],[9,9,6],[18,18,12],[36,36,24]]

In [27]:
import itertools
import numpy as np


def make_rpn_windows(f_shape):
    """
    Generating anchor boxes at each voxel on the feature map,
    the center of the anchor box on each voxel corresponds to center
    on the original input image.

    return
    windows: list of anchor boxes, [z, y, x, d, h, w]
    """
    stride = 4
    anchors = np.asarray(anchors)
    offset = (float(stride) - 1) / 2
    _, _, D, H, W = f_shape
    oz = np.arange(offset, offset + stride * (D - 1) + 1, stride)
    oh = np.arange(offset, offset + stride * (H - 1) + 1, stride)
    ow = np.arange(offset, offset + stride * (W - 1) + 1, stride)

    windows = []
    for z, y, x, a in itertools.product(oz, oh , ow , anchors):
        windows.append([z, y, x, a[0], a[1], a[2]])
    windows = np.array(windows)

    return windows

def find_nearest(x,y,z):
    stride = 4
    start = (stride-1) / 2
    z_stride = 4
    z_start = (z_stride-1) / 2
    xl = ((x - start) // stride) *stride + start
    xr = xl+stride
    xn = xl if x-xl < xr-x else xr

    yl = ((y - start) // stride) *stride + start
    yr = yl+stride
    yn = yl if y-yl < yr-y else yr

    zl = ((z - z_start) // z_stride) *z_stride + z_start
    zr = zl+z_stride
    zn = zl if z-zl < zr-z else zr

    return xn, yn, zn

def IoU(cord1, shape, cord2, shape2):
    x1, y1, z1 = cord1
    w, h, d = shape      # nodule bbox shape
    x2, y2, z2 = cord2
    aw, ah, ad = shape2  # anchor shape

    x_overlap = min(x1+w/2, x2+aw/2) - max(x1-w/2, x2-aw/2) if (max(x1+w/2, x2+aw/2)-min(x1-w/2, x2-aw/2)) < w+aw else 0
    y_overlap = min(y1+h/2, y2+ah/2) - max(y1-h/2, y2-ah/2) if (max(y1+h/2, y2+ah/2)-min(y1-h/2, y2-ah/2)) < h+ah else 0
    z_overlap = min(z1+d/2, z2+ad/2) - max(z1-d/2, z2-ad/2) if (max(z1+d/2, z2+ad/2)-min(z1-d/2, z2-ad/2)) < d+ad else 0
    v = x_overlap * y_overlap*z_overlap
    return v / (w*h*d+aw*ah*ad-v)

def distance(cord1, cord2):
    x1, y1, z1 = cord1
    x2, y2, z2 = cord2
    return ((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)

def DIoU(cord1, shape, cord2, shape2):
    x1, y1, z1 = cord1
    w, h, d = shape      # nodule bbox shape
    x2, y2, z2 = cord2
    aw, ah, ad = shape2  # anchor shape

    c1 = [max(x1+w/2, x2+aw/2), max(y1+h/2, y2+ah/2), max(z1+d/2, z2+ad/2)]
    c2 = [min(x1-w/2, x2-aw/2), min(y1-h/2, y2-ah/2), min(z1-d/2, z2-ad/2)]

    return IoU(cord1, shape, cord2, shape2) - distance(cord1, cord2)/distance(c1, c2)

In [93]:
import json
nodule_distribution = [0,0,0,0]
anchor_distribution = [0 for i in range(len(anchors))]

train_txt = r'F:\master\code\Lung_Nodule\FL_lung_nodule_datasplit_ME_LDCT\client0_train.txt'
with open(train_txt, 'r') as f:
    train_files = f.readlines()
anchor_count = [0,0]
bad_nodule = []
for train_file in train_files[1:]:
    p1, p2 = train_file.replace('\n','').split(',')

    dicom_nodule_path = os.path.join(p1, 'mask', f'{p2}_nodule_count.json') # CHEST1001_nodule_count
    with open(dicom_nodule_path, 'r') as f:
        dicom_nodule = json.load(f)
    nodules = dicom_nodule['bboxes']
    good_nodule = 0
    for nodule in nodules:
        cx = (nodule[0][0]+nodule[1][0])/2
        cy = (nodule[0][1]+nodule[1][1])/2
        cz = (nodule[0][2]+nodule[1][2])/2
        w = (nodule[1][0]-nodule[0][0])+4
        h = (nodule[1][1]-nodule[0][1])+4
        d = (nodule[1][2]-nodule[0][2])+4

        size = (4/3)*3.1415*(h*w*d / 6)
        if size <= 52:
            size_idx = 0
        elif size <= 113:
            size_idx = 1
        elif size <= 268:
            size_idx = 2
        else:
            size_idx = 3

        nodule_distribution[size_idx] += 1

        nearest_point = find_nearest(cx, cy, cz)

        for i, anchor in enumerate(anchors):
            if IoU([cx,cy,cz], [w,h,d], nearest_point, anchor) >= 0.3:
                anchor_distribution[i] += 1
                good_nodule = 1

        anchor_count[good_nodule] += 1
        if not good_nodule:
            bad_nodule.append([size_idx, w,h,d])
print(anchor_distribution, '\n', nodule_distribution, '\n', anchor_count)

[329, 1491, 76, 1387, 615, 36] 
 [0, 3, 46, 1761] 
 [1, 1809]


In [95]:
a = '217.5776	324.21124	182.56894	7.231955	7.961705	5.010294'
b = '217.84135	317.79977	182.8455	8.86131	8.294744	5.4044294'
a = a.split('\t')
a_cord = [float(a_) for a_ in a[:3]]
a_shape = [float(a_)+2. for a_ in a[3:]]
b = b.split('\t')
b_cord = [float(b_) for b_ in b[:3]]
b_shape = [float(b_)+2. for b_ in b[3:]]
IoU(a_cord, a_shape, b_cord, b_shape)

0.19259244358604574

In [96]:
a_shape, b_shape

([9.231955, 9.961705, 7.010294], [10.86131, 10.294744, 7.4044294])

In [41]:
sub_path = r'F:\master\code\LSSANet-main\MsaNet_R_results\ME_LDCT\AdamW0.0003_Bs6x4_OHEM10_bb0\res\72\FROC\submission_rpn.csv'
gt_path  = r'F:\master\code\Lung_Nodule\FL_lung_nodule_datasplit_ME_LDCT\client0_test_annotation.csv'

import csv
sub_by_file = {}
with open(sub_path, newline='') as fs:
    subs = csv.reader(fs,)
    for sub in subs:
        if sub[0] in sub_by_file.keys():
            sub_by_file[sub[0]].append([float(xyzhwd) for xyzhwd in sub[1:8]])
        elif sub[0] != 'series_id':
            sub_by_file[sub[0]] = [[float(xyzhwd) for xyzhwd in sub[1:8]]]

gt_by_file  = {}
with open(gt_path, newline='') as fg:
    gts = csv.reader(fg)
    for gt in gts:
        if gt[0] in gt_by_file.keys():
            gt_by_file[gt[0]].append([float(xyzhwd) for xyzhwd in gt[1:8]])
        elif gt[0] != 'series_id':
            gt_by_file[gt[0]] = [[float(xyzhwd) for xyzhwd in gt[1:8]]]

files = list(gt_by_file.keys())


In [26]:
TP = []
FN = []
gt_IoU = []
gt_DIoU = []
for file in files:
    gts = gt_by_file[file]
    subs = sub_by_file[file]
    for gt in gts:
        is_cand = 0
        re_detected = 0
        max_iou = 0.0
        max_diou = 0.0
        gt_cord, gt_shape = gt[:3], gt[3:6]
        for sub in subs:
            iou = IoU(gt_cord, gt_shape, sub[:3], sub[3:6])
            if iou >= 0.1:
                if is_cand:
                    re_detected += 1
                is_cand = 1
            max_iou = max(max_iou, iou)
            max_diou = max(max_diou, DIoU(gt_cord, gt_shape, sub[:3], sub[3:6]))
        gt_IoU.append([max_iou, max_diou])
        gt_DIoU.append(max_diou)
        if is_cand:
            TP.append(max_iou)
        else:
            FN.append(max_iou)
                

In [45]:
import os
import numpy as np
import warnings
import json
from single_config import config
from scipy.ndimage import zoom

def crop_with_lobe(z:str, yx:str):
    zlobe = z.replace('\n','').split(',')
    Ds, De = [int(z_) for z_ in zlobe]
    yxlobe = yx.replace('\n','').split(',')
    Hs, He, Ws, We = [int(yx_) for yx_ in yxlobe]
    
    # align to 16
    align = 16
    Ds = (Ds//align)*align
    De = ((De+align-1)//align)*align
    Hs = (Hs//align)*align
    He = ((He+align-1)//align)*align
    Ws = (Ws//align)*align
    We = ((We+align-1)//align)*align

    return (Ds, De, Hs, He, Ws, We)

def load_img(filename):
    path, dir = filename.split(',')
    img = np.load('%s\\npy\\%s.npy' % (path, dir))
    # img = img[np.newaxis,...] # (y, x, z) -> (1, y, x, z)
    ## load lobe info
    with open('%s\\npy\\lobe_info.txt' %(path)) as f:
        lobe_info = f.readlines()[-2:]
    with open('%s\\mask\\%s_nodule_count.json' % (path, dir)) as f:
        nodule_count = json.load(f)
    bboxes = nodule_count['bboxes']
    Ds, De, Hs, He, Ws, We = crop_with_lobe(*lobe_info)
    # crop the lobe
    img = img[np.newaxis, Hs:He, Ws:We, Ds:De]

    img = np.clip(img, -1000, 400)
    img = img.astype(np.float32)
    img = img.transpose(0, 3, 1, 2) # (1, y, x, z) -> (1, z, y, x)
    images = img + 1000    # 0 ~ max
    images = (images - 700) / 700 # -1 ~ 1
    return images, (Ds, De, Hs, He, Ws, We), bboxes

class Crop(object):
    def __init__(self, config):
        self.crop_size = config['crop_size']
        self.bound_size = config['bound_size']
        self.stride = config['stride']
        self.pad_value = config['pad_value']

    def __call__(self, imgs, target, bboxes, lobe=None, isScale=False, isRand=False):
        '''
        img: 3D image loading from npy, (1, d, h, w)
        target: one nodule
        bboxes: all nodules in series
        '''
        if isScale:
            radiusLim = [8.,120.]
            scaleLim = [0.75,1.25]
            scaleRange = [np.min([np.max([(radiusLim[0]/target[3]),scaleLim[0]]),1])
                         ,np.max([np.min([(radiusLim[1]/target[3]),scaleLim[1]]),1])]
            scale = np.random.rand()*(scaleRange[1]-scaleRange[0])+scaleRange[0]
            crop_size = (np.array(self.crop_size).astype('float')/scale).astype('int')
        else:
            crop_size = self.crop_size
        bound_size = self.bound_size
        target = np.copy(target)
        bboxes = np.copy(bboxes)
        start = []
        for i in range(3):
            # start.append(int(target[i] - crop_size[i] / 2))
            if not isRand:
                # crop the sample base on target
                r = target[i+3]/2
                s = np.floor(target[i] - r) + 1 - bound_size
                e = np.ceil(target[i] + r) + 1 + bound_size - crop_size[i]
            else:
                # crop the sample randomly
                s = np.max([imgs.shape[i+1]-crop_size[i]/2, imgs.shape[i+1]/2+bound_size])
                e = np.min([crop_size[i]/2, imgs.shape[i+1]/2-bound_size])
                target = np.array([np.nan, np.nan, np.nan, np.nan])
            if s > e:
                i_start = np.random.randint(e, s)
                i_start = max(min(i_start, imgs.shape[i+1]-crop_size[i]),0)
                start.append(i_start)#!
            else:
                start.append(int(target[i])-crop_size[i]/2+np.random.randint(-bound_size/2,bound_size/2))

        coord=[]
        pad = []
        pad.append([0,0])

        for i in range(3):
            leftpad = max(0,-start[i]) # how many pixel need to pad on the left side
            rightpad = max(0,start[i]+crop_size[i]-imgs.shape[i+1]) # how many pixel need to pad on the right side
            pad.append([leftpad,rightpad])
            
        crop = imgs[:,
            int(max(start[0],0)):int(min(start[0] + int(crop_size[0]),imgs.shape[1])),
            int(max(start[1],0)):int(min(start[1] + int(crop_size[1]),imgs.shape[2])),
            int(max(start[2],0)):int(min(start[2] + int(crop_size[2]),imgs.shape[3]))]

        crop = np.pad(crop, pad, 'constant', constant_values=0.67142)
        for i in range(3):
            target[i] = target[i] - start[i]
        for i in range(len(bboxes)):
            for j in range(3):
                bboxes[i][j] = bboxes[i][j] - start[j]

        if isScale:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                crop = zoom(crop, [1, scale, scale, scale], order=1)
            newpad = self.crop_size[0] - crop.shape[1:][0]
            if newpad < 0:
                crop = crop[:, :-newpad, :-newpad, :-newpad]
            elif newpad > 0:
                pad2 = [[0, 0], [0, newpad], [0, newpad], [0, newpad]]
                crop = np.pad(crop, pad2, 'constant', constant_values=0.67142)
            for i in range(6):
                target[i] = target[i] * scale
            for i in range(len(bboxes)):
                for j in range(6):
                    bboxes[i][j] = bboxes[i][j] * scale

        return crop, target, bboxes, coord

crop = Crop(config)

In [ ]:
test_txt = r'F:\master\code\Lung_Nodule\FL_lung_nodule_datasplit_ME_LDCT\client0_test.txt'
with open(test_txt, 'r') as f:
    files = f.readlines()
imgs, lobe_info, bboxes = load_img(files[1])
bboxes = bboxes - [lobe_info[0], lobe_info[2], lobe_info[4], 0, 0, 0]
for bbox in bboxes:
    sample, target, bboxes, coord = crop(imgs, bbox, bboxes, isScale=False, isRand=False)
    

In [123]:
import os
import json
import numpy as np
from utils.util import crop_with_lobe
test_dicom_npy = r'F:\master\code\Lung_Nodule\dataset\LDCT_test_dataset\CHESTCT_Test0014\npy\CHESTCT_Test0014.npy'
test_dicom_info = r'F:\master\code\Lung_Nodule\dataset\LDCT_test_dataset\CHESTCT_Test0014\mask\CHESTCT_Test0014_nodule_count.json'
test_dicom_lobe = r'F:\master\code\Lung_Nodule\dataset\LDCT_test_dataset\CHESTCT_Test0014\npy\lobe_info.txt'

filename = [r'F:\master\code\Lung_Nodule\dataset\LDCT_test_dataset\CHESTCT_Test0014', 'CHESTCT_Test0014']
path, dir = filename
img = np.load(os.path.join('%s\\npy\\%s.npy' % (path, dir)))
img = img[np.newaxis,...] # (y, x, z) -> (1, y, x, z)
## load lobe info
with open(os.path.join('%s\\npy\\lobe_info.txt' %(path))) as f:
    # ['z_start,z_end\n',
    #  'y_start,y_end,x_start,x_end']
    lobe_info = f.readlines()[-2:]
    lobes = crop_with_lobe(*lobe_info)

with open(test_dicom_info, 'r') as f:
    dicom_info = json.load(f)
    nodules = dicom_info["bboxes"]
bboxes = []
for nodule in nodules:
    nodule = np.array(nodule)
    cx, cy, cz = (nodule[1,:] + nodule[0,:]+1) /2
    w,  h,  d  = (nodule[1,:] - nodule[0,:]+1)
    bboxes.append(np.array([0.,cz, cy, cx, d, h, w]))
bboxes = np.array(bboxes)

In [121]:
lobes

(16, 288, 128, 400, 64, 448)

In [124]:
bboxes

array([[  0. ,  71.5, 171. , 332.5,   5. ,   6. ,   7. ],
       [  0. , 251.5, 407. , 335. ,   9. ,  10. ,  12. ],
       [  0. , 141. , 185.5, 368.5,   6. ,   7. ,   5. ]])

In [138]:
from utils.pybox import *
import torch
while True:
    overlap_nodule = False
    rz = np.random.randint(lobes[0]+64, lobes[1]-64)
    ry = np.random.randint(lobes[2]+64, lobes[3]-64)
    rx = np.random.randint(lobes[4]+64, lobes[5]-64)
    overlaps = torch_overlap(bboxes[:,-6:], np.array([rz, ry, rx, 128, 128, 128]))
    for overlap in overlaps:
        if overlap[0] > 0.00:
            overlap_nodule = True
            break
    if not overlap_nodule:
        break
    else:
        print(f'{rz} {ry} {rx} overlapped with nodule')
print(rz, ry, rx)



87 214 364 overlapped with nodule
81 248 223


In [73]:
Ds, De, Hs, He, Ws, We = crop_with_lobe(*lobe_info)
# crop the lobe
img = img[np.newaxis, Hs:He, Ws:We, Ds:De]

img = np.clip(img, -1000, 400)
img = img.astype(np.float32)
img = img.transpose(0, 3, 1, 2) # (1, y, x, z) -> (1, z, y, x)
images = img + 1000    # 0 ~ max
images = (images - 700) / 700 # -1 ~ 1

ValueError: axes don't match array

In [126]:
rz = np.random.randint(lobes[0]+64, lobes[1]-64)
ry = np.random.randint(lobes[2]+64, lobes[3]-64)
rx = np.random.randint(lobes[4]+64, lobes[5]-64)
print(rz, ry, rx)
torch_overlap(bboxes[:,-6:], np.array([rx, ry, rz, 128, 128,128]))

146 308 317


tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])

In [183]:
a = np.array([i for i in range(10)])
p = np.array([0.1 for i in range(10)])

In [184]:
count = np.array([0 for i in range(10)])
for i in range(1000):
    idcs = np.random.choice(len(a), 2, replace=False)
    count[idcs] += 1

random choice

In [185]:
count

array([201, 217, 185, 203, 194, 188, 193, 223, 218, 178])

In [186]:
count2 = np.array([0 for i in range(10)])
for i in range(1000):
    idcs = np.random.choice(len(a), 2, replace=False, p=p)
    p[idcs] *= 0.5
    p /= p.sum()
    count2[idcs]+=1

choice with weight p

In [187]:
count2

array([198, 201, 200, 199, 200, 200, 200, 201, 200, 201])

In [192]:
count.var()

209.0

In [193]:
count2.var()

0.8

In [81]:
import numpy as np
decay = 0.9
w = np.array([0.25,0.25,0.25,0.25])
p = 0.75
pos = 0
neg = 0
for i in range(1000):
    idx = np.random.choice(4,1,p=w)
    if idx <= 2:
        pos +=1
    else:
        neg +=1
        # w *= decay
    w[idx] *= decay
    w /= w.sum()

print(pos, neg)

749 251


In [51]:
w

1.2345679012345678

In [104]:
np.random.rand()

0.049309694441802776

In [2]:
from dataset.bbox_reader_neg_nodulebased import BboxReader_NegNB
from single_config import config, datasets_info
data_dir = r'F:\master\code\Lung_Nodule\dataset\ME_dataset'
set_name = r'F:\master\code\Lung_Nodule\FL_lung_nodule_datasplit_ME_LDCT\client0_train.txt'
augtype = config['augtype']
dataset = BboxReader_NegNB(data_dir, set_name, augtype, config, mode='train')


In [3]:
from torch.utils.data import DataLoader
from dataset.collate import train_collate
train_loader = DataLoader(dataset, batch_size=6, shuffle=True,
                              num_workers=6, pin_memory=True, collate_fn=train_collate, drop_last=True)

In [ ]:
for j, (input, truth_box, truth_label) in enumerate(train_loader):
    print(truth_label[:,0])

In [22]:
import numpy as np 

if np.random.choice(4, 1, p=[0.25,0.25,0.25,0.25]) < 2:
    print('True')

In [26]:
int(np.round(439.49))

439

In [38]:
tpr = 61.9647355163728
tnr = 99.99930071969138
tol_pos = 1191
tol_neg = 60633768
recall = tpr/100
precision = (tpr*tol_pos) / (tpr*tol_pos + (100-tnr)*tol_neg)
f1 = 2*recall*precision/(recall+precision)
print(recall,'\n',precision,'\n',f1)

0.6196473551637279 
 0.6351118760755053 
 0.6272843178919424


In [45]:
from evaluationScript.tools import csvTools
anno_path = r'F:\master\code\LSSANet-main\MsaNet_R_results\ME_LDCT\AdamW0.0003_Bs6x4_fOHEM10_bb0_nbneg\res\72\FROC\submission_rpn.csv'
annotations = csvTools.readCSV(anno_path)

In [97]:
dicom_nodules = {}
for anno in annotations[1:]:
    sid, x, y, z, w, h, d, prob = anno
    if sid in dicom_nodules.keys():
        dicom_nodules[sid] = np.append(dicom_nodules[sid],np.array([[float(x),float(y),float(z),float(w),float(h),float(d), float(prob)]]),axis=0)
    else:
        dicom_nodules[sid] = np.array([[float(x),float(y),float(z),float(w),float(h),float(d), float(prob)]])

In [98]:
dicom_nodules['F:\\master\\code\\Lung_Nodule\\dataset\\ME_dataset\\CHEST1102\\mask\\CHEST1102_nodule_count.json']

array([[ 0.        ,  0.        ,  0.        , 11.679075  , 10.78574   ,
         7.5788956 ,  0.9593786 ],
       [ 0.        ,  0.        ,  0.        ,  5.172542  ,  4.8349504 ,
         3.7945857 ,  0.65252703],
       [ 0.        ,  0.        ,  0.        , 12.183861  , 14.153741  ,
         7.0153294 ,  0.63018936],
       [ 0.        ,  0.        ,  0.        ,  5.922808  ,  5.99413   ,
         4.313506  ,  0.5809836 ],
       [ 0.        ,  0.        ,  0.        ,  5.778343  ,  5.52759   ,
         4.060829  ,  0.5738536 ],
       [ 0.        ,  0.        ,  0.        ,  6.453224  ,  6.188073  ,
         4.099965  ,  0.5635797 ],
       [ 0.        ,  0.        ,  0.        ,  6.79552   ,  6.665095  ,
         4.5868487 ,  0.5495065 ]])

In [99]:
from utils.pybox import torch_nms
import torch
test = dicom_nodules['F:\master\code\Lung_Nodule\dataset\ME_dataset\CHESTCT1747\mask\CHESTCT1747_nodule_count.json'] + [0,0,0,2,2,2,0]
out,_ = torch_nms(torch.from_numpy(test.astype(np.float32)), 0.1)

In [100]:
out

tensor([[ 0.0000,  0.0000,  0.0000, 10.8613, 10.2947,  7.4044,  0.9506],
        [ 0.0000,  0.0000,  0.0000,  9.7671, 10.4456,  7.5393,  0.9115],
        [ 0.0000,  0.0000,  0.0000, 13.1707, 13.3566,  9.7485,  0.8455],
        [ 0.0000,  0.0000,  0.0000,  7.9193,  7.4513,  6.1191,  0.6550],
        [ 0.0000,  0.0000,  0.0000,  9.0956,  8.8516,  6.4882,  0.5422]])

In [89]:
torch.from_numpy(test.astype(np.float32))

tensor([[217.8414, 317.7998, 182.8455,  10.8613,  10.2947,   7.4044,   0.9506],
        [217.8274, 317.9329, 187.8406,   9.7671,  10.4456,   7.5393,   0.9115],
        [153.9819, 306.9145,  79.3042,  13.1707,  13.3566,   9.7485,   0.8455],
        [206.7219, 178.6786, 178.4819,  13.7577,  14.5464,   9.9385,   0.6905],
        [162.8256, 177.9178, 219.1056,   7.9193,   7.4513,   6.1191,   0.6550],
        [217.5776, 324.2112, 182.5689,   9.2320,   9.9617,   7.0103,   0.6323],
        [182.0679, 310.7646,  67.2307,   9.9825,   9.6879,   7.0060,   0.5791],
        [182.0941, 310.5662,  71.1550,   9.0956,   8.8516,   6.4882,   0.5422],
        [162.8644, 177.9783, 215.4896,   7.6482,   7.4470,   6.0287,   0.5159]])